# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

H. Beuther  ->  H. Beuther  |  ['H. Beuther']
J. Lian  ->  J. Lian  |  ['J. Lian']
N. Storm  ->  N. Storm  |  ['N. Storm']
G. Guiglion  ->  G. Guiglion  |  ['G. Guiglion']
M. Bergemann  ->  M. Bergemann  |  ['M. Bergemann']
A. Frank  ->  A. Frank  |  ['A. Frank']
A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
Arxiv has 56 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2308.00993


extracting tarball to tmp_2308.00993...

 done.


Found 159 bibliographic references in tmp_2308.00993/cygnus.bbl.
Retrieving document from  https://arxiv.org/e-print/2308.01111


...

extracting tarball to tmp_2308.01160... done.
Retrieving document from  https://arxiv.org/e-print/2308.01230


extracting tarball to tmp_2308.01230...

 done.


/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3508: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3508: LatexWarning: Found documentclass in tmp_2308.01230/Infant_BHs.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'authors.tex' from 'tmp_2308.01230/authors.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'authors.tex' from 'tmp_2308.01230/authors.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/arxiv_

Found 169 bibliographic references in tmp_2308.01230/Infant_BHs.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2308.00993-b31b1b.svg)](https://arxiv.org/abs/arXiv:2308.00993) | **A global view on star formation: The GLOSTAR Galactic plane survey VIII.  Formaldehyde absorption in Cygnus~X**  |
|| Y. Gong, et al. -- incl., <mark>H. Beuther</mark> |
|*Appeared on*| *2023-08-03*|
|*Comments*| *27 pages, 23 figures, accepted for publication in A&A*|
|**Abstract**| Cygnus X is one of the closest and most active high-mass star-forming regions in our Galaxy, making it one of the best laboratories for studying massive star formation. As part of the GLOSTAR Galactic plane survey, we performed large scale simultaneous H$_{2}$CO (1$_{1,0}$-1$_{1,1}$) spectral line and radio continuum imaging observations toward Cygnus X at $\lambda\sim$6 cm with the Karl G. Jansky Very Large Array and the Effelsberg-100 m radio telescope. Our Effelsberg observations reveal widespread H$_{2}$CO (1$_{1,0}$-1$_{1,1}$) absorption with a spatial extent of $\gtrsim$50 pc in Cygnus~X for the first time. On large scales of 4.4 pc, the relative orientation between local velocity gradient and magnetic field tends to be more parallel at H$_{2}$ column densities of $\gtrsim$1.8$\times 10^{22}$~cm$^{-2}$. On the smaller scale of 0.17 pc, our VLA+Effelsberg combined data reveal H$_{2}$CO absorption only toward three bright H{\scriptsize II} regions. Our observations demonstrate that H$_{2}$CO (1$_{1,0}$-1$_{1,1}$) is commonly optically thin. Kinematic analysis supports the assertion that molecular clouds generally exhibit supersonic motions on scales of 0.17-4.4 pc. We show a non-negligible contribution of the cosmic microwave background radiation in producing extended absorption features in Cygnus X. Our observations suggest that H$_{2}$CO ($1_{1,0}-1_{1,1}$) can trace molecular gas with H$_{2}$ column densities of $\gtrsim 5 \times 10^{21}$ cm$^{-2}$. The ortho-H$_{2}$CO fractional abundance with respect to H$_{2}$ has a mean value of 7.0$\times 10^{-10}$. A comparison of velocity dispersions on different linear scales suggests that the dominant $-3$ km s$^{-1}$ velocity component in the prominent DR21 region has nearly identical velocity dispersions on scales of 0.17-4.4 pc, which deviates from the expected behavior of classic turbulence. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2308.01111-b31b1b.svg)](https://arxiv.org/abs/arXiv:2308.01111) | **Observational constraints on the origin of the elements. VI. Origin and  evolution of neutron-capture elements as probed by the Gaia-ESO survey**  |
|| <mark>J. Lian</mark>, et al. -- incl., <mark>N. Storm</mark>, <mark>G. Guiglion</mark>, <mark>M. Bergemann</mark> |
|*Appeared on*| *2023-08-03*|
|*Comments*| *14 pages, 11 figures, accepted by MNRAS*|
|**Abstract**| Most heavy elements beyond the iron peak are synthesized via neutron capture processes. The nature of the astrophysical sites of neutron capture processes is still very unclear. In this work we explore the observational constraints of the chemical abundances of s-process and r-process elements on the sites of neutron-capture processes by applying Galactic chemical evolution (GCE) models to the data from Gaia-ESO large spectroscopic stellar survey. For the r-process, the [Eu/Fe]-[Fe/H] distribution suggests a short delay time of the site that produces Eu. Other independent observations (e.g., NS-NS binaries), however, suggest a significant fraction of long delayed ($>1$Gyr) neutron star mergers (NSM). When assuming NSM as the only r-process sites, these two observational constraints are inconsistent at above 1$\sigma$ level. Including short delayed r-process sites like magneto-rotational supernova can resolve this inconsistency. For the s-process, we find a weak metallicity dependence of the [Ba/Y] ratio, which traces the s-process efficiency. Our GCE model with up-to-date yields of AGB stars qualitatively reproduces this metallicity dependence, but the model predicts a much higher [Ba/Y] ratio compared to the data. This mismatch suggests that the s-process efficiency of low mass AGB stars in the current AGB nucleosynthesis models could be overestimated. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2308.01230-b31b1b.svg)](https://arxiv.org/abs/arXiv:2308.01230) | **JADES. The diverse population of infant Black Holes at 4<z<11: merging,  tiny, poor, but mighty**  |
|| R. Maiolino, et al. -- incl., <mark>A. d. Graaff</mark> |
|*Appeared on*| *2023-08-03*|
|*Comments*| *Submitted to A&A, 25 pages, 13 figures, 4 tables*|
|**Abstract**| We present 12 new AGN at 4<z<7 in the JADES survey (in addition to the previously identified AGN in GN-z11 at z=10.6) revealed through the detection of a Broad Line Region as seen in the Balmer emission lines. The depth of JADES, together with the use of three different spectral resolutions, enables us to probe a lower mass regime relative to previous studies. In a few cases we find evidence for two broad components of Halpha which suggests that these could be candidate merging black holes (BHs). The inferred BH masses range between 8 x 10^7 Msun down to 4 x 10^5 Msun, interestingly probing the regime expected for Direct Collapse Black Holes. The inferred AGN bolometric luminosities (~10^44-10^45 erg/s) imply accretion rates that are < 0.5 times the Eddington rate in most cases. However, small BH, with M_BH ~ 10^6 Msun, tend to accrete at Eddington or super-Eddington rates. These BH at z~4-11 are over-massive relative to their host galaxies stellar masses when compared to the local M_BH-Mstar relation. However, we find that these early BH tend to be more consistent with the local relation between M_BH and velocity dispersion, as well as between M_BH and dynamical mass, suggesting that these are more fundamental and universal relations. On the BPT excitation-diagnostic diagram these AGN are located in the region that is that is locally occupied by star-forming galaxies, implying that they would be missed by the standard classification techniques if they did not display broad lines. Their location on the diagram is consistent with what expected for AGN hosted in metal poor galaxies (Z ~ 0.1-0.2 Zsun). The fraction of broad line AGN with L_AGN > 10^44 erg/s, among galaxies in the redshift range 4<z<6, is about 10%, suggesting that the contribution of AGN and their hosts to the reionization of the Universe is > 10%. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2308.01160-b31b1b.svg)](https://arxiv.org/abs/arXiv:2308.01160) | **The Oxygen Bottleneck for Technospheres**  |
|| A. Balbi, <mark>A. Frank</mark> |
|*Appeared on*| *2023-08-03*|
|*Comments*| *10 pages, 2 figures*|
|**Abstract**| On Earth, the development of technology required easy access to open air combustion, which is only possible when oxygen partial pressure, P(O$_2$), is above 18\%. This suggests that only planets with significant atmospheric oxygen concentrations will be capable of developing ``advanced'' technospheres and hence detectable technosignatures. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2308.00993.md
    + _build/html/tmp_2308.00993/./fig/peak-abs.png
    + _build/html/tmp_2308.00993/./fig/DR21-VE.png
    + _build/html/tmp_2308.00993/./fig/DR22-VE.png
    + _build/html/tmp_2308.00993/./fig/G76-VE.png
    + _build/html/tmp_2308.00993/./fig/peaktau.png
exported in  _build/html/2308.01111.md
    + _build/html/tmp_2308.01111/./new_figures_ns/ls-hs-omega-zconst_v3.png
    + _build/html/tmp_2308.01111/./ls-hs-yields-check.png
    + _build/html/tmp_2308.01111/./eufe-feh-nsm-omega.png
exported in  _build/html/2308.01230.md
    + _build/html/tmp_2308.01230/./clara_gemma.png
    + _build/html/tmp_2308.01230/./mbh_gal.png
    + _build/html/tmp_2308.01230/./mbh_ledd.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\Teff}{T_{\rm{eff}}}$
$\newcommand{\kms}{km s^{-1}}$
$\newcommand{\Wmq}{Wm^{-2}}$
$\newcommand{\ergps}{erg s^{-1}}$
$\newcommand{\mum}{ \mum}$
$\newcommand{◦ee}{^{\circ}}$
$\newcommand{\Msun}{M_{\odot}}$
$\newcommand{\Lsun}{L_{\odot}}$
$\newcommand{\Msunyr}{ M_{\odot}yr^{-1}}$
$\newcommand{\OI}{[O I]}$
$\newcommand{\SII}{[S II]}$
$\newcommand{\NII}{[N II]}$
$\newcommand{\CaII}{Ca II}$
$\newcommand{\HeI}{He I}$
$\newcommand{\LiI}{Li I}$
$\newcommand{\nodata}{...}$
$\newcommand{\accunit}{M_{\odot} yr^{-1}}$
$\newcommand{\rev}{ }$
$\newcommand{\newrev}{ }$
$\newcommand{\newnewrev}{\bf }$
$\newcommand{\fig}{Fig.}$
$\newcommand{\Mjup}{M_{\rm{Jup}}}$
$\newcommand{\Mdotacc}{\dot M_{\rm{acc}}}$
$\newcommand{\gisela}[1]{{\bf \color{black}[#1]}}$</div>



<div id="title">

# A global view on star formation: The GLOSTAR Galactic plane survey

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2308.00993-b31b1b.svg)](https://arxiv.org/abs/2308.00993)<mark>Appeared on: 2023-08-03</mark> -  _27 pages, 23 figures, accepted for publication in A&A_

</div>
<div id="authors">

Y. Gong, et al. -- incl., <mark>H. Beuther</mark>

</div>
<div id="abstract">

**Abstract:** Cygnus X is one of the closest and most active high-mass star-forming regions in our Galaxy, making it one of the best laboratories for studying massive star formation. We aim to investigate the properties of molecular gas structures on different linear scales with 4.8 GHz formaldehyde (H $_{2}$ CO) absorption line in Cygnus X. As part of the GLOSTAR Galactic plane survey, we performed large scale (7◦ee $\times$ 3◦ee) simultaneous H $_{2}$ CO (1 $_{1,0}$ --1 $_{1,1}$ ) spectral line and radio continuum imaging observations toward Cygnus X at $\lambda\sim$ 6 cm with the Karl G. Jansky Very Large Array and the Effelsberg-100 m radio telescope. We used auxiliary HI, $^{13}$ CO (1--0), dust continuum, and dust polarization data for our analysis. Our Effelsberg observations reveal widespread H $_{2}$ CO (1 $_{1,0}$ --1 $_{1,1}$ ) absorption with a spatial extent of $\gtrsim$ 50 pc in Cygnus X for the first time. On large scales of 4.4 pc, the relative orientation between local velocity gradient and magnetic field tends to be more parallel at H $_{2}$ column densities of $\gtrsim$ 1.8 $\times 10^{22}$ cm $^{-2}$ . On the smaller scale of 0.17 pc, our VLA+Effelsberg combined data reveal H $_{2}$ CO (1 $_{1,0}$ --1 $_{1,1}$ ) absorption only toward three bright H ${\scriptsize II}$ regions. Our observations demonstrate that H $_{2}$ CO (1 $_{1,0}$ --1 $_{1,1}$ ) is commonly optically thin. Kinematic analysis supports the assertion that molecular clouds generally exhibit supersonic motions on scales of 0.17--4.4 pc. We show a non-negligible contribution of the cosmic microwave background radiation in producing extended absorption features in Cygnus X. Our observations suggest that H $_{2}$ CO ( $1_{1,0}-1_{1,1}$ ) can trace molecular gas with H $_{2}$ column densities of $\gtrsim 5 \times 10^{21}$ cm $^{-2}$ (i.e., $A_{\rm V} \gtrsim 5$ ). The ortho-H $_{2}$ CO fractional abundance with respect to H $_{2}$ has a mean value of 7.0 $\times 10^{-10}$ .A comparison of velocity dispersions on different linear scales suggests that the dominant $-3$  $\kms$ velocity component in the prominent DR21 region has nearly identical velocity dispersions on scales of 0.17--4.4 pc, which deviates from the expected behavior of classic turbulence.

</div>

<div id="div_fig1">

<img src="tmp_2308.00993/./fig/peak-abs.png" alt="Fig10" width="100%"/>

**Figure 10. -** {(a) Effelsberg 4.89 GHz radio continuum emission overlaid with the peak absorption contours of H$_{2}$CO (1$_{1,0}$--1$_{1,1}$). The corresponding HPBW of H$_{2}$CO (1$_{1,0}$--1$_{1,1}$) is 3$\arcmin$. The color bar represents the flux densities of the radio continuum emission. The H$_{2}$CO absorption contours start from $-$0.5 K (5$\sigma$), and decrease by 0.5 K. The developed H{\scriptsize II} regions from [Anderson, Bania and Balser (2014)]() are marked with red solid circles, while SNR G78.2+2.1 is indicated by the orange dashed circle. Blue crosses represent the radio continuum sources and active star-forming objects, and the purple star represents the massive star cluster, Cygnus OB2. (b) Similar to Fig. \ref{Fig:peak-abs}a but the corresponding HPBW of H$_{2}$CO (1$_{1,0}$--1$_{1,1}$) is 10$\rlap{.}$\arcmin8. The H$_{2}$CO absorption contours start from $-$0.08 K (4$\sigma$), and decrease by 0.06 K. In both panels, the beam size is shown in the lower left corner.} (*Fig:peak-abs*)

</div>
<div id="div_fig2">

<img src="tmp_2308.00993/./fig/DR21-VE.png" alt="Fig15.1" width="33%"/><img src="tmp_2308.00993/./fig/DR22-VE.png" alt="Fig15.2" width="33%"/><img src="tmp_2308.00993/./fig/G76-VE.png" alt="Fig15.3" width="33%"/>

**Figure 15. -** {_Top_: Observed H$_{2}$CO (1$_{1,0}$--1$_{1,1}$) spectra of DR21 (a), DR22 (c), and G76.1883+0.0973 (e) overlaid on the fit results indicated by the brown dashed lines. The derived optical depth spectra are shown by the red lines. In panels (a), (c), (e), the black dashed vertical lines represent the LSR velocities of the H{\scriptsize II} regions obtained from the radio recombination line measurements (Khan et al. in prep). _Bottom_: VLA+Effelsberg 4.9 GHz radio continuum emission of DR21 (b), DR22 (d), and G76.1883+0.0973 (f) overlaid with the H$_{2}$CO (1$_{1,0}$--1$_{1,1}$) absorption contours. For DR21, the blue and red contours represent the H$_{2}$CO (1$_{1,0}$--1$_{1,1}$) absorption peak for the $-$3 $\kms$ and 8 $\kms$ components, respectively. The contours start at $-$0.1 Jy beam$^{-1}$ (5$\sigma$), with each subsequent contour being twice the previous one. For DR22 and G76.1883+0.0973, the contours start at $-$0.1 Jy beam$^{-1}$ (5$\sigma$) and decrease by 0.04 Jy beam$^{-1}$. The synthesized beam is shown in the lower left corner of each panel. All the continuum and spectral line data are from the combination of the VLA D configuration and the Effelsberg single-dish observations.} (*Fig:vla-image*)

</div>
<div id="div_fig3">

<img src="tmp_2308.00993/./fig/peaktau.png" alt="Fig11" width="100%"/>

**Figure 11. -** {(a) Distribution of the peak optical depth of the H$_{2}$CO (1$_{1,0}$--1$_{1,1}$) line. The HPBW of the H$_{2}$CO image is 3$\arcmin$. The color bar represents the peak optical depth. The H{\scriptsize II} regions from [Anderson, Bania and Balser (2014)]() are marked with red solid circles, while SNR G78.2+2.1 is indicated by an orange dashed circle. The blue crosses represent the radio continuum sources and active star-forming objects, and the purple star represents the massive star cluster, Cygnus OB2. (b) Similar to Fig. \ref{Fig:peaktau}a but the  HPBW of the H$_{2}$CO (1$_{1,0}$--1$_{1,1}$) image is 10$\rlap{.}$\arcmin8. (c) Similar to Fig. \ref{Fig:peaktau}b but for the H$_{2}$CO column density in the 1$_{1,0}$ level.
In all panels, the beam size is shown in the lower left corner.} (*Fig:peaktau*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2308.00993"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\keywords}[1]$
$\newcommand{\jl}{\textcolor{orange}}$
$\newcommand{\mb}{\textcolor{red}}$
$\newcommand{\as}{\textcolor{pink}}$</div>



<div id="title">

# Observational constraints on the origin of the elements. VI. Origin and evolution of neutron-capture elements as probed by the Gaia-ESO survey

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2308.01111-b31b1b.svg)](https://arxiv.org/abs/2308.01111)<mark>Appeared on: 2023-08-03</mark> -  _14 pages, 11 figures, accepted by MNRAS_

</div>
<div id="authors">

<mark>J. Lian</mark>, et al. -- incl., <mark>N. Storm</mark>, <mark>G. Guiglion</mark>, <mark>M. Bergemann</mark>

</div>
<div id="abstract">

**Abstract:** Most heavy elements beyond the iron peak are synthesized via neutron capture processes . The nature of the ${astrophysical}$ sites of neutron capture processes ${is}$ still very unclear.In this work we explore the observational constraints of the chemical abundances of s-process and r-process elements on the sites of neutron-capture processes by applying ${Galactic}$ chemical evolution (GCE) models to the data from Gaia-ESO ${large spectroscopic stellar}$ survey. For the r-process, the [ Eu/Fe ] - [ Fe/H ] distribution suggests a short delay time of the site that produces Eu. Other independent observations (e.g., NS-NS binaries), however, suggest a significant fraction of long delayed ( $>1$ Gyr) neutron star mergers (NSM). When assuming NSM as the only r-process sites, these two observational constraints are inconsistent at above 1 $\sigma$ level. Including short delayed r-process sites like ${magneto-rotational}$ supernova can resolve this inconsistency. For the s-process, we find a weak metallicity dependence of ${the}$ [ Ba/Y ] ratio,  which traces the s-process efficiency. Our GCE model with up-to-date yields of AGB stars qualitatively reproduces this metallicity dependence, ${but the model predicts a much higher [Ba/Y] ratio compared to the data.}$ This mismatch suggests that the s-process efficiency of low mass AGB stars in the current AGB nucleosynthesis ${models could be}$ overestimated.

</div>

<div id="div_fig1">

<img src="tmp_2308.01111/./new_figures_ns/ls-hs-omega-zconst_v3.png" alt="Fig4" width="100%"/>

**Figure 4. -** Relative abundance ratio between heavy {(Ba) and light (Y)} s-process elements as a function of [Fe/H]. Four GCE models with three test models deviating from the basic model in three aspects are included, one without rotating CCSN (blue dashed), one without AGB contribution (purple dotted), and one with AGB yields from Karakas et al. (magenta dash-dotted). (*ls-hs-omega*)

</div>
<div id="div_fig2">

<img src="tmp_2308.01111/./ls-hs-yields-check.png" alt="Fig8" width="100%"/>

**Figure 8. -** Heavy-to-light s-process abundance ratio ([hs/ls]) parameterized by the ratio [Ba/Y] in AGB yields from \citet{cristallo2015}(left panel) and Karakas et al. (right panel). Different masses of AGB stars are shown in different colours. {See text}. (*ls-hs-agb*)

</div>
<div id="div_fig3">

<img src="tmp_2308.01111/./eufe-feh-nsm-omega.png" alt="Fig9" width="100%"/>

**Figure 9. -** [Eu/Fe]-[Fe/H] evolution predicted by GCE models with NSM as the only sites of r-process. NSM DTD is characterised by the maximum delay time ($\tau_{\rm max}$) and the slope of power-law DTD ($\alpha$). Different NSM DTDs with different $\tau_{\rm max}$(left) and $\alpha$(right) are assumed as illustrated in the legend. (*eufe-feh-nsm-omega*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2308.01111"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\todo}[1]{\textcolor{orange}{[to do: #1]}}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\kms}{\ensuremath{\mathrm{km s^{-1}}}\xspace}$
$\newcommand{Å}{\oldAA\xspace}$</div>



<div id="title">

# JADES. The diverse population of infant Black Holes at 4$<$z$<$11: merging, tiny, poor, but mighty

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2308.01230-b31b1b.svg)](https://arxiv.org/abs/2308.01230)<mark>Appeared on: 2023-08-03</mark> -  _Submitted to A&A, 25 pages, 13 figures, 4 tables_

</div>
<div id="authors">

R. Maiolino, et al. -- incl., <mark>A. d. Graaff</mark>

</div>
<div id="abstract">

**Abstract:** Spectroscopy with the James Webb Space Telescope has opened the possibility to identify moderate luminosity Active Galactic Nuclei (AGN) in the early Universe, at and beyond the epoch of reionization, complementing previous surveys of much more luminous (and much rarer) quasars. We present 12 new AGN at 4 $<$ z $<$ 7 in the JADES survey (in addition to the previously identified AGN in GN-z11 at z=10.6) revealed through the detection of a Broad Line Region as seen in the Balmer emission lines. The depth of JADES, together with the use of three different spectral resolutions,  enables us to probe a lower mass regime relative to previous studies. In a few cases we find evidence for two broad components of  H $\alpha$ , which suggests that these could be candidate merging black holes (BHs). The inferred BH masses range between $\rm 8\times 10^7  M_\odot$ down to $\rm 4\times 10^5  M_\odot$ , interestingly probing the regime expected for Direct Collapse Black Holes. The inferred AGN bolometric luminosities ( $\sim 10^{44}-10^{45}$ erg/s) imply accretion rates that are $< 0.5$ times the Eddington rate in most cases. However, small BHs, with $\rm M_{BH} \sim 10^6 M_\odot$ , tend to accrete at Eddington or super-Eddington rates. These BHs at z $\sim$ 4-11 are over-massive relative to their host galaxies stellar masses when compared to the local $\rm M_{BH}-M_{star}$ relation. However, we find that these early BH tend to be more consistent with the local relation between $\rm M_{BH}$ and velocity dispersion, as well as between $\rm M_{BH}$ and dynamical mass, suggesting that these are more fundamental and universal relations. On the BPT excitation-diagnostic diagram these AGN are located in the region that is that is locally occupied by star-forming galaxies, implying that they would be missed by the standard classification techniques if they did not display broad lines. Their location on the diagram is consistent with what expected for AGN hosted in metal poor galaxies ( $\rm Z \sim 0.1-0.2 Z_\odot$ ).The fraction of broad line AGN with $\rm L_{AGN}>10^{44} erg/s$ among galaxies in the redshift range $4<z<6$ is about 10 \% , suggesting that the contribution of AGN $_ and_$ their hosts to the reionization of the Universe is $>$ 10 \% .

</div>

<div id="div_fig1">

<img src="tmp_2308.01230/./clara_gemma.png" alt="Fig3" width="100%"/>

**Figure 3. -** 
Spectra around H$\alpha$ of ID 10013704, the highest redshift AGN showing indication for a dual BLR.
The top and bottom panels show the medium and high resolution spectra, respectively. The line coding is the same as in Fig. \ref{fig:spectra}, but in this case the violet dashed line shows the intermediate broad component that is needed to  reproduce the observed profile. The central and rightmost panels show the spectrum from which the narrow components, as well as one of the two broad components have been removed, to better highlight the significance of the other broad component.
 (*fig:clara_gemma*)

</div>
<div id="div_fig2">

<img src="tmp_2308.01230/./mbh_gal.png" alt="Fig7" width="100%"/>

**Figure 7. -** 
BH mass as a function of host galaxy properties, specifically: stellar mass (top-left),  velocity dispersion (bottom-left), dynamical mass (bottom-right). The JADES results presented in this work are shown with large golden circles. Orange dashed vertical lines connect candidate merging BHs. Blue symbols indicate measurements from other JWST surveys at high-z, as indicated in the legend (only detections at $>3\sigma$ are shown). Gray triangles are measurements of high-z QSOs using ALMA data.  Small red symbols show the distribution of local galaxies as indicated in the legend; straight lines show the local relation fits (with shaded regions providing the scatter and slope uncertainty).
In the top-left panel small orange circles show the  maximum stellar masses estimated for a few JADES AGN (see text), while in the bottom-left panel small squares indicate the effect of correcting for rotation velocity broadening within the slit.
 (*fig:mbh_host*)

</div>
<div id="div_fig3">

<img src="tmp_2308.01230/./mbh_ledd.png" alt="Fig1" width="100%"/>

**Figure 1. -** 
Distribution of black hole masses and Eddington ratios (L/L$_{\rm Edd}$) for the broad line AGN in JADES (large golden circles). The orange dashed lines connect candidate dual AGN. We also show the results from other JWST surveys with blue symbols (see legend). Note that the apparent anticorrelation is probably spurious, as the BH mass is at the denominator of the Y-axis quantity. The plot has simply the purpose of visually illustrating the distribution of the two quantities.
 (*fig:mbh_ledd*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2308.01230"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

245  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

11  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
